In [1]:
import os; os.chdir('..')
import numpy as np
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import FloatSlider

In [2]:
from prev_ob_models.Birgiolas2020.isolated_cells import MC1 as MC

In [3]:
mc = MC()

In [4]:
from neuron import h

vec = h.Vector()
vec.record(mc.cell.soma(0.5)._ref_v, 0.5)


h.tstop = 10000

#h.newPlotV()

In [5]:
ic = h.IClamp(mc.cell.soma(0.5))
ic2 = h.IClamp(mc.cell.soma(0.5))
ic3 = h.IClamp(mc.cell.soma(0.5))

In [9]:
ic.delay = 1000
ic.dur = 3000
ic.amp = -0.111


ic2.delay = 4000
ic2.dur = 2000
ic2.amp = 0.151


ic3.delay = 7000
ic3.dur = 2000
ic3.amp = 0.111*2

In [7]:
from neuron import gui

In [8]:
h.newPlotV()
h.run()
raise

TypeError: exceptions must be old-style classes or derived from BaseException, not NoneType

In [ ]:
params = [    
    { "start": 1,  "attr": "diam",        "low": 0.1, "high": 10.0, "lists": ["apical","basal","axonal"] },
    { "start": 34.77,  "attr": "Ra",        "low": 5.0, "high": 100.0, "lists": ["all"] },
    { "start": 2.706,   "attr": "cm",        "low": 0.1, "high": 4.0, "lists": ["all"] },
    { "start": 49.95,  "attr": "ena",       "low": 40.0, "high": 50.0, "lists": ["all"] },
    { "start": -70.03,  "attr": "ek",        "low": -100.0, "high": -70.0, "lists": ["all"] },
    { "start": -64.42,  "attr": "e_pas",     "low": -70.0, "high": -50.0, "lists": ["all"] },
    { "start": 0.0005955, "attr": "g_pas",     "low": 0, "high": 0.00004, "lists": ["all"] },    
    { "start": 0.5955,  "attr": "sh_Na",     "low": 0, "high": 10, "lists": ["all"] },    
    { "start": 10,  "attr": "tau_CaPool",     "low": 1, "high": 20, "lists": ["all"] },    

    { "start":  0.87485,  "attr": "gbar_Na",     "low": 0, "high": 0.05, "lists": ["all"] },
    { "start": 0.0297,  "attr": "gbar_Kd",     "low": 0, "high": 0.05, "lists": ["all"] },
    { "start": 0.000264,  "attr": "gbar_Kslow",  "low": 0, "high": 0.008, "lists": ["all"] },
    { "start": 0.07215,  "attr": "gbar_KA",     "low": 0, "high": 0.005, "lists": ["all"] },
    { "start": 0.001,  "attr": "gbar_KCa",    "low": 0, "high": 0.004, "lists": ["all"] },
    { "start": 0.00081441,  "attr": "gbar_LCa",  "low": 0, "high": 0.001, "lists": ["all"] },
    
    { "start": -30.805,  "attr": "eh",       "low": -40.0, "high": -25.0, "lists": ["apical"] },
    { "start": 0.00335,  "attr": "gbar_Ih",    "low": 0, "high": 0.000003, "lists": ["apical"] },
    { "start": 0.000107,  "attr": "gbar_CaT",    "low": 0, "high": 18e-3, "lists": ["apical"] },
]

In [ ]:
best = mc.param_values

for i, p in enumerate(params):
    params[i]["start"] = best[i]

In [ ]:
kwargs = {}
for pi, pv in enumerate(params):
    #print(pv["attr"])
    kwargs[pv["attr"]+"|"+str(pv["lists"])] = FloatSlider(
        value=pv["start"],
        min=pv["low"],
        max=pv["high"],
        step=(pv["high"]-pv["low"])/100.0,
        continuous_update=False,
        readout_format='.9f',
        description=(pv["attr"]+"|"+str(pv["lists"][0])).replace("gbar_","g")
    )
#kwargs

In [ ]:
def evaluate(**kwargs):   
    param_keys = kwargs.keys()
    
    for key in param_keys:
        attr = key.split("|")[0]
        lists = eval(key.split("|")[1])
        
        for param_list in lists:
            for sec in getattr(mc.cell, param_list):
                if attr == "tau_CaPool":
                    setattr(h, attr, kwargs[key])
                elif attr == "diam":
                    setattr(sec, attr, kwargs[key]*getattr(sec, attr))
                else:
                    setattr(sec, attr, kwargs[key])
        
    h.run()
    
    from matplotlib import pyplot as plt
    plt.plot(vec.as_numpy())
    plt.show()
    
    return mc.cell.soma(0.5).v

In [ ]:
interact(evaluate, **kwargs);